In [13]:
import numpy as np
import time
import random
import sys
import psutil
import math
from itertools import permutations

In [14]:
# Đọc dữ liệu từ file TSP
def read_tsp_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        
    dimension = 0
    edge_weight_format = ""
    edge_weights = []
    reading_weights = False
    
    for line in lines:
        line = line.strip()
        if line.startswith("DIMENSION"):
            dimension = int(line.split(":")[1].strip())
        elif line.startswith("EDGE_WEIGHT_FORMAT"):
            edge_weight_format = line.split(":")[1].strip()
        elif line.startswith("EDGE_WEIGHT_SECTION"):
            reading_weights = True
            continue
        
        if reading_weights and line != "EOF":
            edge_weights.extend([float(x) for x in line.split()])
    # Khởi tạo ma trận khoảng cách từ định dạng LOWER_DIAG_ROW
    dist_matrix = np.zeros((dimension, dimension))
    
    if edge_weight_format == "LOWER_DIAG_ROW":
        index = 0
        for i in range(dimension):
            for j in range(i + 1):
                dist_matrix[i][j] = edge_weights[index]
                if i != j:  # Nếu không phải đường chéo, gán giá trị đối xứng
                    dist_matrix[j][i] = edge_weights[index]
                index += 1
    
    return dist_matrix


In [15]:
# Hàm tính tổng độ dài của một hành trình
def calculate_tour_distance(tour, dist_matrix):
    distance = 0
    for i in range(len(tour) - 1):
        distance += dist_matrix[tour[i]][tour[i+1]]
    # Thêm quãng đường từ thành phố cuối cùng quay về thành phố đầu tiên
    distance += dist_matrix[tour[-1]][tour[0]]
    return distance

# Hàm đo lượng bộ nhớ sử dụng
def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # MB



In [16]:
# 1. Thuật toán tham lam (Nearest Neighbor)
def nearest_neighbor(dist_matrix):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Bắt đầu từ thành phố 0
    current_city = 0
    unvisited = set(range(1, n))
    tour = [current_city]
    
    # Liên tục chọn thành phố gần nhất chưa thăm
    while unvisited:
        next_city = min(unvisited, key=lambda city: dist_matrix[current_city][city])
        tour.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city
    
    distance = calculate_tour_distance(tour, dist_matrix)
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": tour,
        "distance": distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [17]:
# 2. Thuật toán duyệt cạn (Brute Force)
def brute_force(dist_matrix):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    if n > 11:  # Giới hạn để tránh tràn bộ nhớ
        return {"tour": [], "distance": float('inf'), "time": 0, "memory": 0}
    
    # Thành phố đầu tiên luôn là 0, chỉ hoán vị các thành phố còn lại
    cities = list(range(1, n))
    best_distance = float('inf')
    best_tour = None
    
    for perm in permutations(cities):
        tour = [0] + list(perm)
        distance = calculate_tour_distance(tour, dist_matrix)
        if distance < best_distance:
            best_distance = distance
            best_tour = tour
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": best_tour,
        "distance": best_distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [18]:
# 3. Thuật toán xấp xỉ (2-opt)
def two_opt(dist_matrix):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Bắt đầu với một tour ban đầu
    tour = list(range(n))
    best_distance = calculate_tour_distance(tour, dist_matrix)
    
    improved = True
    while improved:
        improved = False
        for i in range(1, n-1):
            for j in range(i+1, n):
                if j - i == 1: continue  # Bỏ qua nếu cạnh liền kề
                
                # Tạo tour mới bằng cách đảo ngược đoạn từ i đến j
                new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
                new_distance = calculate_tour_distance(new_tour, dist_matrix)
                
                if new_distance < best_distance:
                    tour = new_tour
                    best_distance = new_distance
                    improved = True
                    break
            if improved: break
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": tour,
        "distance": best_distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [19]:
# 4. Thuật toán bầy kiến (Ant Colony Optimization)
def ant_colony_optimization(dist_matrix, n_ants=10, n_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, Q=100):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Khởi tạo ma trận pheromone
    pheromone = np.ones((n, n))
    best_tour = None
    best_distance = float('inf')
    
    for iteration in range(n_iterations):
        all_tours = []
        all_distances = []
        
        # Mỗi con kiến tạo một hành trình
        for ant in range(n_ants):
            tour = [random.randint(0, n-1)]  # Bắt đầu từ một thành phố ngẫu nhiên
            unvisited = set(range(n))
            unvisited.remove(tour[0])
            
            # Xây dựng hành trình
            while unvisited:
                current = tour[-1]
                probabilities = []
                
                for city in unvisited:
                    tau = pheromone[current][city]
                    eta = 1.0 / (dist_matrix[current][city] + 1e-10)  # Tránh chia cho 0
                    probabilities.append((city, tau**alpha * eta**beta))
                
                # Chọn thành phố tiếp theo dựa trên xác suất
                total = sum(prob[1] for prob in probabilities)
                r = random.random() * total
                cum_prob = 0
                next_city = None
                
                for city, prob in probabilities:
                    cum_prob += prob
                    if cum_prob >= r:
                        next_city = city
                        break
                
                if next_city is None:  # Trường hợp làm tròn số
                    next_city = probabilities[-1][0]
                
                tour.append(next_city)
                unvisited.remove(next_city)
            
            distance = calculate_tour_distance(tour, dist_matrix)
            all_tours.append(tour)
            all_distances.append(distance)
            
            if distance < best_distance:
                best_distance = distance
                best_tour = tour
        
        # Cập nhật pheromone: bay hơi
        pheromone *= (1 - evaporation_rate)
        
        # Cập nhật pheromone: bổ sung
        for tour, distance in zip(all_tours, all_distances):
            for i in range(len(tour) - 1):
                pheromone[tour[i]][tour[i+1]] += Q / distance
                pheromone[tour[i+1]][tour[i]] += Q / distance
            # Khép vòng lặp
            pheromone[tour[-1]][tour[0]] += Q / distance
            pheromone[tour[0]][tour[-1]] += Q / distance
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": best_tour,
        "distance": best_distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [20]:
# 5. Thuật toán di truyền (Genetic Algorithm)
def genetic_algorithm(dist_matrix, pop_size=50, n_generations=100, mutation_rate=0.1):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Tạo quần thể ban đầu
    population = []
    for _ in range(pop_size):
        tour = list(range(n))
        random.shuffle(tour)
        population.append(tour)
    
    best_tour = None
    best_distance = float('inf')
    
    for generation in range(n_generations):
        # Đánh giá
        fitness_scores = []
        for tour in population:
            distance = calculate_tour_distance(tour, dist_matrix)
            fitness_scores.append(1.0 / (distance + 1e-10))  # Thể trạng cao hơn cho khoảng cách ngắn hơn
            
            if distance < best_distance:
                best_distance = distance
                best_tour = tour[:]
        
        # Tạo quần thể mới
        new_population = []
        
        # Elitism: giữ lại cá thể tốt nhất
        best_idx = fitness_scores.index(max(fitness_scores))
        new_population.append(population[best_idx])
        
        # Tạo thế hệ mới
        while len(new_population) < pop_size:
            # Chọn cha mẹ
            parents_idx = random.choices(range(pop_size), weights=fitness_scores, k=2)
            parent1, parent2 = population[parents_idx[0]], population[parents_idx[1]]
            
            # Lai ghép (Ordered Crossover)
            child = [-1] * n
            start, end = sorted(random.sample(range(n), 2))
            
            # Sao chép đoạn từ parent1
            for i in range(start, end + 1):
                child[i] = parent1[i]
            
            # Điền phần còn lại từ parent2
            j = 0
            for i in range(n):
                if child[i] == -1:
                    while parent2[j] in child:
                        j += 1
                    child[i] = parent2[j]
                    j += 1
            
            # Đột biến (swap mutation)
            if random.random() < mutation_rate:
                idx1, idx2 = random.sample(range(n), 2)
                child[idx1], child[idx2] = child[idx2], child[idx1]
            
            new_population.append(child)
        
        population = new_population
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": best_tour,
        "distance": best_distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [21]:
# 6. Thuật toán láng giềng (Simulated Annealing)
def simulated_annealing(dist_matrix, temp_initial=1000, temp_final=0.01, cooling_rate=0.95, iterations_per_temp=100):
    n = len(dist_matrix)
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Khởi tạo hành trình
    current_tour = list(range(n))
    random.shuffle(current_tour)
    current_distance = calculate_tour_distance(current_tour, dist_matrix)
    
    best_tour = current_tour[:]
    best_distance = current_distance
    
    # Simulated annealing
    temp = temp_initial
    
    while temp > temp_final:
        for _ in range(iterations_per_temp):
            # Tạo hàng xóm bằng cách hoán đổi hai thành phố
            i, j = sorted(random.sample(range(n), 2))
            
            # Tạo tour mới
            new_tour = current_tour[:]
            # Đảo ngược đoạn từ i đến j
            new_tour[i:j+1] = reversed(new_tour[i:j+1])
            
            new_distance = calculate_tour_distance(new_tour, dist_matrix)
            
            # Tiêu chí chấp nhận Metropolis
            delta = new_distance - current_distance
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_tour = new_tour
                current_distance = new_distance
                
                if current_distance < best_distance:
                    best_tour = current_tour[:]
                    best_distance = current_distance
        
        # Giảm nhiệt độ
        temp *= cooling_rate
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        "tour": best_tour,
        "distance": best_distance,
        "time": end_time - start_time,
        "memory": end_memory - start_memory
    }


In [22]:
def main():
    # Đường dẫn tới file TSP
    tsp_file = "citytest.tsp"
    
    # Đọc dữ liệu
    dist_matrix = read_tsp_file(tsp_file)
    
    # Chạy các thuật toán
    algorithms = {
        "Thuật toán tham lam (Nearest Neighbor)": nearest_neighbor,
        "Thuật toán duyệt cạn (Brute Force)": brute_force,
        "Thuật toán xấp xỉ (2-opt)": two_opt,
        "Thuật toán bầy kiến (Ant Colony Optimization)": ant_colony_optimization,
        "Thuật toán di truyền (Genetic Algorithm)": genetic_algorithm,
        "Thuật toán láng giềng (Simulated Annealing)": simulated_annealing
    }
    
    for name, algorithm in algorithms.items():
        print(f"\n{name}:")
        result = algorithm(dist_matrix)
        print(f"Hành trình: {result['tour']}")
        print(f"Tổng quãng đường: {result['distance']}")
        print(f"Thời gian thực thi: {result['time']:.6f} giây")
        print(f"Dung lượng bộ nhớ sử dụng: {result['memory']:.6f} MB")

if __name__ == "__main__":
    main()


Thuật toán tham lam (Nearest Neighbor):
Hành trình: [0, 1, 9, 8, 5, 6, 4, 7, 2, 3]
Tổng quãng đường: 31.0
Thời gian thực thi: 0.000000 giây
Dung lượng bộ nhớ sử dụng: 0.000000 MB

Thuật toán duyệt cạn (Brute Force):
Hành trình: [0, 1, 9, 3, 8, 5, 6, 4, 7, 2]
Tổng quãng đường: 27.0
Thời gian thực thi: 4.100482 giây
Dung lượng bộ nhớ sử dụng: 0.000000 MB

Thuật toán xấp xỉ (2-opt):
Hành trình: [0, 1, 9, 3, 4, 7, 8, 5, 6, 2]
Tổng quãng đường: 31.0
Thời gian thực thi: 0.000000 giây
Dung lượng bộ nhớ sử dụng: 0.000000 MB

Thuật toán bầy kiến (Ant Colony Optimization):
Hành trình: [3, 8, 5, 6, 4, 7, 2, 0, 1, 9]
Tổng quãng đường: 27.0
Thời gian thực thi: 0.239741 giây
Dung lượng bộ nhớ sử dụng: 0.000000 MB

Thuật toán di truyền (Genetic Algorithm):
Hành trình: [9, 3, 8, 5, 6, 4, 7, 2, 0, 1]
Tổng quãng đường: 27.0
Thời gian thực thi: 0.130667 giây
Dung lượng bộ nhớ sử dụng: 0.000000 MB

Thuật toán láng giềng (Simulated Annealing):
Hành trình: [8, 5, 6, 4, 7, 2, 0, 1, 9, 3]
Tổng quãng đường: 2